##asd

In [1]:
import requests
import urllib.request
import time
import json
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [2]:
def get_link_pages(user):
    pages = []
    url = 'https://letterboxd.com/' + user + '/watchlist/by/rating/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    lis = soup('li', attrs={'class': 'paginate-page'})
    for n in range(2, 10000):
        try:
            url = 'https://letterboxd.com/' + user + '/watchlist/by/rating/page/' + str(n)
            response = requests.get(url)
            page_soup = BeautifulSoup(response.text, 'html.parser')
            poster_container = page_soup('li', attrs={'class': "poster-container"})
            if poster_container:
                pages.append(url)
            else:
                break
        except:
            break
    return pages



def ObtenerURLs(users):
    urls = []
    for user in users:
        pages = []
        url = 'https://letterboxd.com/' + user + '/watchlist/by/rating/'
        pages.append(url)
        added_pages = get_link_pages(user)
        for page in added_pages:
            pages.append(page)
        urls.append(pages)

    urls_dict = {}
    for user in users:
        n = users.index(user)
        urls_dict[user] = urls[n]
    return urls_dict
    #     values = {}
    #     values['URLs'] = pages
    #     data[user] = values
    # return data
    
def ObtenerPelis(urls_dict):
    all_films = {}
    for user, links in urls_dict.items():
        values = []
        for link in links:
            response = requests.get(link)
            soup = BeautifulSoup(response.text, 'html.parser')
            films = soup.findAll('img', attrs={'class': 'image'})
            for item in films:
                values.append(str(item['alt']).encode('utf-8').decode())
        all_films[user] = values

    return all_films


def GenerarDataFrame(films): 
    df = pd.DataFrame()
    for key, value in films.items():
        col = pd.DataFrame(data=value, columns=['Pelis'], index=[key for x in range(len(value))])
        df = df.append(col)
    
    # for user, col in df.iteritems():   
    #     df[user].fillna('NO_MATCH', inplace=True)
    return df



In [3]:
users = ['lukigol', 'teficapriata', 'anablasi']
urls = ObtenerURLs(users)

In [4]:
films = ObtenerPelis(urls)

In [5]:
df = GenerarDataFrame(films)

In [6]:
duplics_col = df[df.duplicated(keep=False)]

#duplics_col = duplics_col.groupby(list(duplics_col)).apply(lambda x: tuple(x.index)).tolist()
df1 = (duplics_col.groupby(duplics_col.columns.tolist())
       .apply(lambda x: tuple(x.index))
       .reset_index(name='users'))
print(df1)

                             Pelis                              users
0                  American Psycho           (teficapriata, anablasi)
1                       Collective           (teficapriata, anablasi)
2                          Cruella            (lukigol, teficapriata)
3                 Django Unchained           (teficapriata, anablasi)
4                       GoodFellas           (teficapriata, anablasi)
5           Grave of the Fireflies            (lukigol, teficapriata)
6                     Isle of Dogs                (lukigol, anablasi)
7                          Memento                (lukigol, anablasi)
8                  Nightmare Alley            (lukigol, teficapriata)
9                     Perfect Blue            (lukigol, teficapriata)
10      Portrait of a Lady on Fire            (lukigol, teficapriata)
11             Requiem for a Dream            (lukigol, teficapriata)
12                        Scarface           (teficapriata, anablasi)
13                  

In [7]:
df1['length'] = df1['users'].str.len()
df1.sort_values('length', ascending=False, inplace=True)
df1

,Pelis,users,length
19,Zodiac,"(lukigol, teficapriata, anablasi)",3
14,The Farewell,"(lukigol, teficapriata, anablasi)",3
1,Collective,"(teficapriata, anablasi)",2
18,Zero Dark Thirty,"(lukigol, anablasi)",2
17,Wildlife,"(teficapriata, anablasi)",2
16,The Shawshank Redemption,"(teficapriata, anablasi)",2
15,"The Good, the Bad and the Ugly","(teficapriata, anablasi)",2
13,The Big Short,"(lukigol, anablasi)",2
12,Scarface,"(teficapriata, anablasi)",2
11,Requiem for a Dream,"(lukigol, teficapriata)",2


In [8]:
unique_vals = df1['users'].unique()

for val in unique_vals:
    movies = df1[df1['users']==val]['Pelis']
    if len(val)==len(users):
        print('Las pelis que quieren ver todes son:\n'+ movies.to_string(index=False)+'\n--------------------------')
    else:
        print(f'Las pelis que quieren ver {str(val)[1:-1]} son:\n' + movies.to_string(index=False)+'\n----------------------')

Las pelis que quieren ver todes son:
      Zodiac
The Farewell
--------------------------
Las pelis que quieren ver 'teficapriata', 'anablasi' son:
                    Collective
                      Wildlife
      The Shawshank Redemption
The Good, the Bad and the Ugly
                      Scarface
               American Psycho
                    GoodFellas
              Django Unchained
----------------------
Las pelis que quieren ver 'lukigol', 'anablasi' son:
Zero Dark Thirty
   The Big Short
         Memento
    Isle of Dogs
----------------------
Las pelis que quieren ver 'lukigol', 'teficapriata' son:
       Requiem for a Dream
              Perfect Blue
           Nightmare Alley
    Grave of the Fireflies
                   Cruella
Portrait of a Lady on Fire
----------------------
